In [1]:
# Import libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')


In [2]:
# Load in data
df=pd.read_csv('Data/AviationData.csv', encoding='latin-1', low_memory=False)
df

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88884,20221227106491,Accident,ERA23LA093,2022-12-26,"Annapolis, MD",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,0.0,1.0,0.0,0.0,NaN,NaN,NaN,29-12-2022
88885,20221227106494,Accident,ERA23LA095,2022-12-26,"Hampton, NH",United States,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
88886,20221227106497,Accident,WPR23LA075,2022-12-26,"Payson, AZ",United States,341525N,1112021W,PAN,PAYSON,...,Personal,NaN,0.0,0.0,0.0,1.0,VMC,NaN,NaN,27-12-2022
88887,20221227106498,Accident,WPR23LA076,2022-12-26,"Morgan, UT",United States,NaN,NaN,NaN,NaN,...,Personal,MC CESSNA 210N LLC,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN


In [3]:
#Dropping rows if the make or model is missing.
#We want to identify which plane is the best investment and cant do so if this info is missing.
df.dropna(subset=['Make', 'Model'], inplace=True)
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 88777 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88777 non-null  object 
 1   Investigation.Type      88777 non-null  object 
 2   Accident.Number         88777 non-null  object 
 3   Event.Date              88777 non-null  object 
 4   Location                88725 non-null  object 
 5   Country                 88552 non-null  object 
 6   Latitude                34347 non-null  object 
 7   Longitude               34338 non-null  object 
 8   Airport.Code            50213 non-null  object 
 9   Airport.Name            52746 non-null  object 
 10  Injury.Severity         87798 non-null  object 
 11  Aircraft.damage         85605 non-null  object 
 12  Aircraft.Category       32245 non-null  object 
 13  Registration.Number     87502 non-null  object 
 14  Make                    88777 non-null

In [4]:
# Here we are making the 'Make' column consistent
df['Make']=df['Make'].str.lower()
df['Make'].value_counts()

cessna                27145
piper                 14869
beech                  5371
boeing                 2738
bell                   2722
                      ...  
murray r. gardner         1
st clair                  1
bryant frederick m        1
jack fehling              1
flight star               1
Name: Make, Length: 7575, dtype: int64

In [5]:
# Here we are making the 'Model' column consistent by removing spaces, -, and _.
# Function to process a single value
def process_category(value):
    # Convert to lowercase, remove spaces, dashes, and underscores, and combine all words
    return ''.join(value.lower().replace(' ', '').replace('-', '').replace('_', ''))

df['Model'] = df['Model'].apply(process_category)
df['Model'].value_counts()

152             2367
172             1756
172n            1172
pa28140         1002
172m             829
                ... 
loensloymf80       1
mustangpsid        1
airbikerx40        1
al3cub             1
kr1catfish         1
Name: Model, Length: 8843, dtype: int64

In [6]:
# Here we are creating a new column that combines Make and Model
df['Make/Model'] = df['Make']+" "+df['Model']
df['Make/Model'].value_counts()

cessna 152                    2366
cessna 172                    1753
cessna 172n                   1171
piper pa28140                 1001
cessna 172m                    829
                              ... 
tingle special                   1
ainsworth lancair4pturbine       1
wright avidflyer                 1
james o'bert varieze             1
hammond twinstargyroplane        1
Name: Make/Model, Length: 15844, dtype: int64

In [7]:
# Initial sample of Airplanes before imputation.
df['Aircraft.Category'].value_counts()

Airplane             27580
Helicopter            3435
Glider                 508
Balloon                231
Gyrocraft              173
Weight-Shift           161
Powered Parachute       91
Ultralight              30
Unknown                 14
WSFT                     9
Powered-Lift             5
Blimp                    4
UNK                      2
ULTR                     1
Rocket                   1
Name: Aircraft.Category, dtype: int64

In [8]:
#Here we are seeing if the make/model for a specific aircraft category matches another row where aircraft category is null.

# 1. df['Aircraft.Category'].isna(): 
#This part creates a boolean mask where it checks if the values in the 'Aircraft.Category' column are NaN. 
#It returns True for NaN values and False otherwise.

# 2. df['Make/Model'].isin(df[df['Aircraft.Category'] == 'Airplane']['Make/Model']): 
# This part creates another boolean mask. It checks if the values in the 'Make/Model' 
# column are present in the 'Make/Model' column of rows where 'Aircraft.Category' is equal to 'Airplane'. 
# It returns True for rows where the 'Make/Model' is in the list of 'Make/Model' values for 'Airplane'.

# 3. mask = df['Aircraft.Category'].isna() & ...: 
# This line combines the two boolean masks using the bitwise AND (&) operator. 
# The resulting mask will be True for rows where 'Aircraft.Category' is NaN and the corresponding 'Make/Model' 
# is in the list of 'Make/Model' values for 'Airplane'.



Airplane_mask = df['Aircraft.Category'].isna() & df['Make/Model'].isin(df[df['Aircraft.Category'] == 'Airplane']['Make/Model'])
Helicopter_mask = df['Aircraft.Category'].isna() & df['Make/Model'].isin(df[df['Aircraft.Category'] == 'Helicopter']['Make/Model'])
Glider_mask = df['Aircraft.Category'].isna() & df['Make/Model'].isin(df[df['Aircraft.Category'] == 'Glider']['Make/Model'])
Balloon_mask = df['Aircraft.Category'].isna() & df['Make/Model'].isin(df[df['Aircraft.Category'] == 'Balloon']['Make/Model'])
Gyrocraft_mask = df['Aircraft.Category'].isna() & df['Make/Model'].isin(df[df['Aircraft.Category'] == 'Gyrocraft']['Make/Model'])
WeightShift_mask = df['Aircraft.Category'].isna() & df['Make/Model'].isin(df[df['Aircraft.Category'] == 'Weight-Shift']['Make/Model'])
PoweredParachute_mask = df['Aircraft.Category'].isna() & df['Make/Model'].isin(df[df['Aircraft.Category'] == 'Powered Parachute']['Make/Model'])
Ultralight_mask = df['Aircraft.Category'].isna() & df['Make/Model'].isin(df[df['Aircraft.Category'] == 'Ultralight']['Make/Model'])
Unknown_mask = df['Aircraft.Category'].isna() & df['Make/Model'].isin(df[df['Aircraft.Category'] == 'Unknown']['Make/Model'])
WSFT_mask = df['Aircraft.Category'].isna() & df['Make/Model'].isin(df[df['Aircraft.Category'] == 'WSFT']['Make/Model'])
PoweredLift_mask = df['Aircraft.Category'].isna() & df['Make/Model'].isin(df[df['Aircraft.Category'] == 'Powered-Lift']['Make/Model'])
Blimp_mask = df['Aircraft.Category'].isna() & df['Make/Model'].isin(df[df['Aircraft.Category'] == 'Blimp']['Make/Model'])
UNK_mask = df['Aircraft.Category'].isna() & df['Make/Model'].isin(df[df['Aircraft.Category'] == 'UNK']['Make/Model'])
ULTR_mask = df['Aircraft.Category'].isna() & df['Make/Model'].isin(df[df['Aircraft.Category'] == 'ULTR']['Make/Model'])
Rocket_mask = df['Aircraft.Category'].isna() & df['Make/Model'].isin(df[df['Aircraft.Category'] == 'Rocket']['Make/Model'])

In [9]:
#Here we are applying the condition above, and reclassfying the Aircraft category if there was a match.
df.loc[Airplane_mask, 'Aircraft.Category'] = 'Airplane'
df.loc[Helicopter_mask, 'Aircraft.Category'] = 'Helicopter'
df.loc[Glider_mask, 'Aircraft.Category'] = 'Glider'
df.loc[Balloon_mask, 'Aircraft.Category'] = 'Balloon'
df.loc[Gyrocraft_mask, 'Aircraft.Category'] = 'Gyrocraft'
df.loc[WeightShift_mask, 'Aircraft.Category'] = 'Weight-Shift'
df.loc[PoweredParachute_mask, 'Aircraft.Category'] = 'Powered Parachute'
df.loc[Ultralight_mask, 'Aircraft.Category'] = 'Ultralight'
df.loc[Unknown_mask, 'Aircraft.Category'] = 'Unknown'
df.loc[WSFT_mask, 'Aircraft.Category'] = 'WSFT'
df.loc[PoweredLift_mask, 'Aircraft.Category'] = 'Powered-Lift'
df.loc[Blimp_mask, 'Aircraft.Category'] = 'Blimp'
df.loc[UNK_mask, 'Aircraft.Category'] = 'UNK'
df.loc[ULTR_mask, 'Aircraft.Category'] = 'ULTR'
df.loc[Rocket_mask, 'Aircraft.Category'] = 'Rocket'



# New value count of Airplanes, we were able to classify ~40k new airplanes, that were previously NA
df['Aircraft.Category'].value_counts()

Airplane             69263
Helicopter            7420
Glider                1030
Balloon                435
Unknown                235
Gyrocraft              219
Weight-Shift           162
Powered Parachute       92
Ultralight              38
Powered-Lift            30
WSFT                    11
Blimp                    5
UNK                      2
ULTR                     1
Rocket                   1
Name: Aircraft.Category, dtype: int64

In [10]:
#Investigating the unknown aircraft category by make and model, to find any aircrafts.
unknown_aircraft_df = df[df['Aircraft.Category']=='Unknown']
unknown_aircraft_df['Make/Model'].value_counts()

bell 206l3                                 97
schweizer 269c                             89
bell 412                                   23
mitsubishi mu300                            8
agusta a119                                 5
quad city challenger                        3
rans s7                                     3
amateur construction denneykitfoxmk2        1
robinson helicopter company r44ii           1
parachute icarus crossfire2                 1
sui ry, laurinen janne, kääriä compair8     1
cessna aircraft company 560xl               1
varieze ruatan                              1
quicksilver mxlsportii                      1
Name: Make/Model, dtype: int64

In [11]:
# Identifying airplanes in the unknown aircraft category and classifying them as airplanes.

unknown_condition_airplane=((df['Aircraft.Category'] == 'Unknown') & 
                   ((df['Make/Model'] == 'rans s7') | (df['Make/Model'] == 'cessna aircraft company 560xl') | 
                    (df['Make/Model'] == 'mitsubishi mu300')))
df.loc[unknown_condition_airplane, 'Aircraft.Category'] = 'Airplane'

df['Aircraft.Category'].value_counts()

#Now we can just drop all rows where the aircraft category is unknown since there are no more airplanes in the categorgy.

Airplane             69275
Helicopter            7420
Glider                1030
Balloon                435
Unknown                223
Gyrocraft              219
Weight-Shift           162
Powered Parachute       92
Ultralight              38
Powered-Lift            30
WSFT                    11
Blimp                    5
UNK                      2
ULTR                     1
Rocket                   1
Name: Aircraft.Category, dtype: int64

In [12]:
# Now lets investigate the NaN values in aircraft categorgy.

None_aircraft_df = df[df['Aircraft.Category'].isna()]
None_aircraft_df['Make'].value_counts()

boeing               443
cessna               340
beech                314
piper                265
mcdonnell douglas    132
                    ... 
charles c. gillis      1
riley, donald          1
robert judd            1
gillette               1
hardin                 1
Name: Make, Length: 3982, dtype: int64

In [13]:
#Here we are assigning the variable 'airplane' to well known airplane manufactures that dont have a aircraft classification.

df.loc[(df['Aircraft.Category'].isna()) & (df['Make'] == 'boeing'), 'Aircraft.Category'] = 'Airplane'
df.loc[(df['Aircraft.Category'].isna()) & (df['Make'] == 'cessna'), 'Aircraft.Category'] = 'Airplane'
df.loc[(df['Aircraft.Category'].isna()) & (df['Make'] == 'beech'), 'Aircraft.Category'] = 'Airplane'
df.loc[(df['Aircraft.Category'].isna()) & (df['Make'] == 'beechcraft'), 'Aircraft.Category'] = 'Airplane'
df.loc[(df['Aircraft.Category'].isna()) & (df['Make'] == 'piper'), 'Aircraft.Category'] = 'Airplane'
df.loc[(df['Aircraft.Category'].isna()) & (df['Make'] == 'mcdonnell douglas'), 'Aircraft.Category'] = 'Airplane'
df.loc[(df['Aircraft.Category'].isna()) & (df['Make'] == 'douglas'), 'Aircraft.Category'] = 'Airplane'

In [14]:
#Now we drop all rows in the aircraft category column that don't equal airplane
airplane_df = df[df['Aircraft.Category'] == 'Airplane']
airplane_df

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date,Make/Model
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN,stinson 1083
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996,piper pa24180
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007,cessna 172m
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000,rockwell 112
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980,cessna 501
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88884,20221227106491,Accident,ERA23LA093,2022-12-26,"Annapolis, MD",United States,NaN,NaN,NaN,NaN,...,NaN,0.0,1.0,0.0,0.0,NaN,NaN,NaN,29-12-2022,piper pa28151
88885,20221227106494,Accident,ERA23LA095,2022-12-26,"Hampton, NH",United States,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,bellanca 7eca
88886,20221227106497,Accident,WPR23LA075,2022-12-26,"Payson, AZ",United States,341525N,1112021W,PAN,PAYSON,...,NaN,0.0,0.0,0.0,1.0,VMC,NaN,NaN,27-12-2022,american champion aircraft 8gcbc
88887,20221227106498,Accident,WPR23LA076,2022-12-26,"Morgan, UT",United States,NaN,NaN,NaN,NaN,...,MC CESSNA 210N LLC,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,cessna 210n


In [15]:
filtered_df = airplane_df[airplane_df['Injury.Severity'].isnull()]
filtered_df.loc[:, ['Event.Date','Injury.Severity','Total.Fatal.Injuries','Total.Serious.Injuries','Total.Minor.Injuries','Total.Uninjured','Broad.phase.of.flight','Make/Model']]

,Event.Date,Injury.Severity,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Broad.phase.of.flight,Make/Model
63918,2008-01-03,NaN,0.0,0.0,0.0,0.0,NaN,boeing 737400
63987,2008-01-24,NaN,0.0,0.0,0.0,0.0,NaN,cessna c208bcaravan
64026,2008-02-03,NaN,0.0,0.0,0.0,0.0,NaN,boeing 737
64128,2008-02-28,NaN,0.0,0.0,0.0,0.0,NaN,piper pa34200
64178,2008-03-10,NaN,0.0,0.0,0.0,0.0,NaN,boeing 737400
...,...,...,...,...,...,...,...,...
88832,2022-11-27,NaN,0.0,0.0,0.0,0.0,NaN,cessna 177
88849,2022-12-05,NaN,0.0,0.0,0.0,0.0,NaN,boeing 737
88879,2022-12-18,NaN,0.0,0.0,0.0,0.0,NaN,airbus a330243
88885,2022-12-26,NaN,0.0,0.0,0.0,0.0,NaN,bellanca 7eca


In [16]:
# Here I am checking to make sure there are no injuries recorded in these rows.
#We are good to remove rows where the Injury Severity is null.
filtered_df['Total.Fatal.Injuries'].sum()
filtered_df['Total.Serious.Injuries'].sum()
filtered_df['Total.Minor.Injuries'].sum()
filtered_df['Total.Uninjured'].sum()

0.0

In [17]:
#Drop rows where the Injury Severity is NA
airplane_df =airplane_df.dropna(subset=['Injury.Severity'])
airplane_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69989 entries, 0 to 88888
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                69989 non-null  object 
 1   Investigation.Type      69989 non-null  object 
 2   Accident.Number         69989 non-null  object 
 3   Event.Date              69989 non-null  object 
 4   Location                69948 non-null  object 
 5   Country                 69888 non-null  object 
 6   Latitude                27816 non-null  object 
 7   Longitude               27807 non-null  object 
 8   Airport.Code            42454 non-null  object 
 9   Airport.Name            44526 non-null  object 
 10  Injury.Severity         69989 non-null  object 
 11  Aircraft.damage         67982 non-null  object 
 12  Aircraft.Category       69989 non-null  object 
 13  Registration.Number     69093 non-null  object 
 14  Make                    69989 non-null

In [18]:
# Looking at rows where the fata injury category is na or zero
airplane_df[airplane_df['Total.Fatal.Injuries'].isnull()]
# airplane_df[airplane_df['Total.Fatal.Injuries'] == 0]
# airplane_df[(airplane_df['Total.Fatal.Injuries'] == 0) | (airplane_df['Total.Fatal.Injuries'].isnull())]

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date,Make/Model
5,20170710X52551,Accident,NYC79AA106,1979-09-17,"BOSTON, MA",United States,42.445277,-70.758333,NaN,NaN,...,Air Canada,NaN,NaN,1.0,44.0,VMC,Climb,Probable Cause,19-09-2017,mcdonnell douglas dc9
89,20020917X02486,Accident,NYC82DA022,1982-01-15,"BUTLER, PA",United States,NaN,NaN,3G9,BUTLER ROE,...,NaN,NaN,NaN,NaN,2.0,VMC,Approach,Probable Cause,15-01-1983,cessna 310
260,20020917X02495,Accident,NYC82DA033,1982-02-07,"MARTHAS VINEYARD, MA",United States,NaN,NaN,NaN,MARTHAS VINEYARD,...,NaN,NaN,NaN,NaN,2.0,VMC,Landing,Probable Cause,05-05-2011,bellanca 7kcab
351,20020917X02151,Incident,FTW82IA062,1982-02-19,"HARLINGEN, TX",United States,NaN,NaN,HRL,HARLINGEN IND.AIR PARK,...,Mexicana Airlines,NaN,NaN,NaN,83.0,IMC,Landing,Probable Cause,04-12-2014,boeing b727200
728,20020917X02383,Accident,MIA82DA098,1982-04-01,"ORMOND BEACH, FL",United States,NaN,NaN,OMN,ORMOND BEACH,...,NaN,NaN,NaN,NaN,1.0,VMC,Taxi,Probable Cause,01-04-1983,cessna 172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63904,20080215X00200,Accident,CHI08CA058,2007-12-29,"CRYSTAL FALLS, MI",United States,46.8,-88.36,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,VMC,Landing,Probable Cause,28-02-2008,cessna 172f
63905,20080214X00193,Accident,CHI08CA056,2007-12-29,"ABINGDON, IL",United States,40.799722,-90.438611,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,VMC,Maneuvering,Probable Cause,28-02-2008,ercoupe (eng & research corp.) 415d
63910,20080129X00122,Accident,CHI08CA057,2007-12-30,"ALEXANDRIA, MN",United States,45.866111,-95.394444,AXN,Chandler Field Airport,...,NaN,NaN,NaN,NaN,1.0,VMC,Takeoff,Probable Cause,28-02-2008,lerohl rv8
63911,20080125X00106,Accident,SEA08CA056,2007-12-31,"SANTA ANA, CA",United States,33.675556,-117.868056,SNA,John Wayne - Orange County,...,NaN,NaN,NaN,NaN,2.0,VMC,Landing,Probable Cause,28-02-2008,piper pa12


In [19]:
#Here I am looking at rows where all injury categories are NaN

filtered_df = airplane_df[airplane_df['Total.Fatal.Injuries'].isnull() &
                          airplane_df['Total.Serious.Injuries'].isnull() &
                          airplane_df['Total.Minor.Injuries'].isnull() &
                          airplane_df['Total.Uninjured'].isnull()]
filtered_df.loc[:, ['Event.Date','Injury.Severity','Total.Fatal.Injuries',
                    'Total.Serious.Injuries','Total.Minor.Injuries','Total.Uninjured','Broad.phase.of.flight','Make/Model']]
#Might be best just to drop all of these rows, since we don't know how many people were on the plane.
#For the standing phases of the flight, we can replace total uninjured with average total people for that make/model

,Event.Date,Injury.Severity,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Broad.phase.of.flight,Make/Model
15579,1986-08-10,Incident,NaN,NaN,NaN,NaN,Standing,mcdonnell douglas dc1040
18430,1987-08-16,Incident,NaN,NaN,NaN,NaN,NaN,boeing b767200er
19110,1987-11-07,Incident,NaN,NaN,NaN,NaN,NaN,mcdonnell douglas dc1015
19342,1987-12-16,Unavailable,NaN,NaN,NaN,NaN,NaN,boeing 747124f
20067,1988-04-14,Incident,NaN,NaN,NaN,NaN,NaN,boeing 747
...,...,...,...,...,...,...,...,...
62585,2007-05-25,Unavailable,NaN,NaN,NaN,NaN,NaN,douglas dc3
63009,2007-07-23,Incident,NaN,NaN,NaN,NaN,NaN,airbus a330
63398,2007-09-16,Unavailable,NaN,NaN,NaN,NaN,NaN,piper pa32301
63646,2007-10-28,Incident,NaN,NaN,NaN,NaN,Standing,boeing 757225


In [20]:
# step 1
#I'm dropping rows, where there is no information to how many passangers were on board.  
columns_to_check = ['Total.Fatal.Injuries','Total.Serious.Injuries','Total.Minor.Injuries','Total.Uninjured']
airplane_df.dropna(subset=columns_to_check, how='all', inplace=True)
airplane_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69809 entries, 0 to 88888
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                69809 non-null  object 
 1   Investigation.Type      69809 non-null  object 
 2   Accident.Number         69809 non-null  object 
 3   Event.Date              69809 non-null  object 
 4   Location                69772 non-null  object 
 5   Country                 69711 non-null  object 
 6   Latitude                27810 non-null  object 
 7   Longitude               27801 non-null  object 
 8   Airport.Code            42436 non-null  object 
 9   Airport.Name            44507 non-null  object 
 10  Injury.Severity         69809 non-null  object 
 11  Aircraft.damage         67863 non-null  object 
 12  Aircraft.Category       69809 non-null  object 
 13  Registration.Number     69053 non-null  object 
 14  Make                    69809 non-null

In [21]:
#Now that one of the four 'columns_to_check' has a value in it, We replace those NA values with 0
airplane_df[columns_to_check] = airplane_df[columns_to_check].fillna(0)
airplane_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69809 entries, 0 to 88888
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                69809 non-null  object 
 1   Investigation.Type      69809 non-null  object 
 2   Accident.Number         69809 non-null  object 
 3   Event.Date              69809 non-null  object 
 4   Location                69772 non-null  object 
 5   Country                 69711 non-null  object 
 6   Latitude                27810 non-null  object 
 7   Longitude               27801 non-null  object 
 8   Airport.Code            42436 non-null  object 
 9   Airport.Name            44507 non-null  object 
 10  Injury.Severity         69809 non-null  object 
 11  Aircraft.damage         67863 non-null  object 
 12  Aircraft.Category       69809 non-null  object 
 13  Registration.Number     69053 non-null  object 
 14  Make                    69809 non-null

In [22]:
# Amatuer built column analysis.
# First I will investigate the NA values in this column, and if the make/model matches another rows make and model, replace NA value.

no_amatuer_mask = airplane_df['Amateur.Built'].isna() & airplane_df['Make/Model'].isin(airplane_df[airplane_df['Amateur.Built'] == 'No']['Make/Model'])
yes_amatuer_mask = airplane_df['Amateur.Built'].isna() & airplane_df['Make/Model'].isin(airplane_df[airplane_df['Amateur.Built'] == 'Yes']['Make/Model'])
airplane_df.loc[no_amatuer_mask, 'Amateur.Built'] = 'No'
airplane_df.loc[yes_amatuer_mask, 'Amateur.Built'] = 'Yes'

# airplane_df[airplane_df['Amateur.Built'].isnull()] 
#Only 3 NAs left, we can drop these rows as these are not planes we are interested in investigating.


# Now I'm gonna remove all rows where Amatuer built is = yes or null
condition = (airplane_df['Amateur.Built'] == 'Yes') | airplane_df['Amateur.Built'].isnull()
airplane_df = airplane_df[~condition]



In [23]:
#Checking to make sure we have No as the only option.
airplane_df['Amateur.Built'].value_counts()

No    66259
Name: Amateur.Built, dtype: int64

In [24]:
airplane_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66259 entries, 0 to 88888
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                66259 non-null  object 
 1   Investigation.Type      66259 non-null  object 
 2   Accident.Number         66259 non-null  object 
 3   Event.Date              66259 non-null  object 
 4   Location                66222 non-null  object 
 5   Country                 66161 non-null  object 
 6   Latitude                24849 non-null  object 
 7   Longitude               24843 non-null  object 
 8   Airport.Code            40048 non-null  object 
 9   Airport.Name            42057 non-null  object 
 10  Injury.Severity         66259 non-null  object 
 11  Aircraft.damage         64327 non-null  object 
 12  Aircraft.Category       66259 non-null  object 
 13  Registration.Number     65525 non-null  object 
 14  Make                    66259 non-null

In [25]:
#Now lets investigate the Aircraft.damage column.

airplane_df['Aircraft.damage'].unique() #['Destroyed', 'Substantial', 'Minor', nan, 'Unknown']
airplane_df['Aircraft.damage'].value_counts() #Substantial=48728 , Destroyed=13541, Minor=1984, Unknown=74
# airplane_df[airplane_df['Aircraft.damage'].isnull()] #1932 rows that are NAN, 


# airplane_df[((airplane_df['Total.Fatal.Injuries'] == 0) &
#     (airplane_df['Total.Serious.Injuries'] == 0) &
#     (airplane_df['Total.Minor.Injuries'] == 0) &
#     (airplane_df['Total.Uninjured'] > 0) &
#     airplane_df['Aircraft.damage'].isna()
# )] #985 rows where the above conditions are true.

Substantial    48728
Destroyed      13541
Minor           1984
Unknown           74
Name: Aircraft.damage, dtype: int64

In [26]:

# airplane_df[airplane_df['Aircraft.damage'] == 'Substantial'] # There are some rows where there are only unijured entries.
#Therefor3 it is not okay to assume that the aircraft was minor damage if there are only uninjured passangers
airplane_df[airplane_df['Aircraft.damage'] == 'Destroyed']
# airplane_df[airplane_df['Aircraft.damage'] == 'Minor']
# airplane_df[airplane_df['Aircraft.damage'] == 'Unknown']


,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date,Make/Model
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN,stinson 1083
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996,piper pa24180
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,NaN,3.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,26-02-2007,cessna 172m
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000,rockwell 112
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,NaN,1.0,2.0,0.0,0.0,VMC,Approach,Probable Cause,16-04-1980,cessna 501
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88794,20221118106319,Accident,CEN23FA038,2022-11-18,"Decatur, TX",United States,331143N,0973410W,NaN,NaN,...,REGIONAL AVIATION LLC,1.0,0.0,0.0,0.0,VMC,NaN,NaN,14-12-2022,cessna 421c
88806,20221122106339,Accident,GAA23WA029,2022-11-21,"Medellin,",Colombia,NaN,NaN,SKMD,NaN,...,Aeropaca S.A.S.,8.0,0.0,0.0,0.0,NaN,NaN,NaN,29-11-2022,piper pa31350
88843,20221205106408,Accident,CEN23FA057,2022-12-04,"Cleburne, TX",United States,321926N,0972553W,CPT,CLEBURNE RGNL,...,NaN,2.0,0.0,0.0,0.0,IMC,NaN,NaN,15-12-2022,cessna 2105(205)
88845,20221205106417,Accident,CEN23FA059,2022-12-05,"Corning, IA",United States,405957N,0944518W,CRZ,Corning Municipal Airport,...,PORTER STEVEN B,1.0,0.0,0.0,0.0,VMC,NaN,NaN,13-12-2022,engineering & research ercoupe415c


In [27]:
#Initial hypothesis: If everyone onboard dies, can we classify the Aircraft damage as Destroyed?
airplane_df[((airplane_df['Total.Fatal.Injuries'] > 0) &
    (airplane_df['Total.Serious.Injuries'] == 0) &
    (airplane_df['Total.Minor.Injuries'] == 0) &
    (airplane_df['Total.Uninjured'] == 0) &
    ~airplane_df['Aircraft.damage'].isna()
)]['Aircraft.damage'].value_counts()

#Here I am showing that even when only looking at cases where everyone dies. We still cannot conclude the damage type.
#Therefor imputation will not work.

Destroyed      8047
Substantial    2284
Unknown          14
Minor             6
Name: Aircraft.damage, dtype: int64

In [28]:
#Dropping rows where the aircraft damage is null or unknown. We should drop 2,006 rows.
airplane_df = airplane_df.dropna(subset=['Aircraft.damage'])
airplane_df = airplane_df[airplane_df['Aircraft.damage'] != 'Unknown']
airplane_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 64253 entries, 0 to 88886
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                64253 non-null  object 
 1   Investigation.Type      64253 non-null  object 
 2   Accident.Number         64253 non-null  object 
 3   Event.Date              64253 non-null  object 
 4   Location                64219 non-null  object 
 5   Country                 64161 non-null  object 
 6   Latitude                24164 non-null  object 
 7   Longitude               24157 non-null  object 
 8   Airport.Code            39181 non-null  object 
 9   Airport.Name            41179 non-null  object 
 10  Injury.Severity         64253 non-null  object 
 11  Aircraft.damage         64253 non-null  object 
 12  Aircraft.Category       64253 non-null  object 
 13  Registration.Number     63589 non-null  object 
 14  Make                    64253 non-null

In [29]:
airplane_df['Make'].value_counts()

cessna                      26698
piper                       14654
beech                        5258
boeing                       1358
mooney                       1269
                            ...  
believer                        1
powrachute                      1
martin                          1
bush caddy usa                  1
piper/wally's flyers inc        1
Name: Make, Length: 1099, dtype: int64

In [30]:
#Common trainer aircraft used in flight schools
#Four seater: Cessna 172, Piper PA-28 Cherokee, 
#Two seaters: cessna 150, cessna 152,  

# airplane_df[airplane_df['Make'].str.contains('piper', case=False, na=False)] #14,722 instances

# airplane_df[airplane_df['Make'].str.contains('cessna', case=False, na=False)] #26,756 instances

#Here I am making sure the make values for pipper and cessna is consistent throught the dataframe.

airplane_df.loc[airplane_df['Make'].str.contains('piper', case=False, na=False), 'Make'] = 'piper'
airplane_df.loc[airplane_df['Make'].str.contains('cessna', case=False, na=False), 'Make'] = 'cessna'

airplane_df['Make'].value_counts()


cessna                         26756
piper                          14722
beech                           5258
boeing                          1358
mooney                          1269
                               ...  
chamberlain gerald                 1
freeman heritage collection        1
believer                           1
powrachute                         1
keuthan                            1
Name: Make, Length: 1074, dtype: int64

In [31]:
#the cessna 172 is also commonly known as “The Cessna Skyhawk”. 
#Replacing the model with 172 if Skyhawk occurs in the model number.

airplane_df.loc[airplane_df['Model'].str.contains('Skyhawk', case=False, na=False), 'Model'] = '172'



In [32]:

airplane_df[airplane_df['Model'].str.contains('pa28', case=False, na=False)]['Model'].value_counts().sum()


# airplane_df[airplane_df['Make'] == 'piper']['Model'].value_counts()
# airplane_df[airplane_df['Make'] == 'cessna']['Model'].value_counts()

4808

In [33]:
#For cessna we are interested in model numbers, 172, 150, and 152
#For pipper we are interested in model numbers, pa-28, 

airplane_df[airplane_df['Make'] == 'piper']['Model'].value_counts()
airplane_df[airplane_df['Make'] == 'cessna']['Model'].str.contains('152', case=False, na=False).value_counts()

False    24259
True      2497
Name: Model, dtype: int64

In [34]:
# Here I am looking creating a new column to classify all versions of the Pipper pa-28, cessna 152, cessna 150, and cessna 170
# so that we can aggregate each model together.
# We will leave the original model column unchanged, becuase maybe we will want to see the difference between the
#cessna 150n vs cessna 150m


# Replace 'make_column', 'model_column', and 'new_column' with the actual column names
make_column = 'Make'
model_column = 'Model'
new_column = 'model_new'

# Create a new column 'model_new' based on conditions
airplane_df[new_column] = airplane_df.apply(
    lambda row: 'pa-28' if row[make_column] == 'piper' and 'pa28' in row[model_column].lower() else (
        '152' if row[make_column] == 'cessna' and '152' in row[model_column] else (
            '172' if row[make_column] == 'cessna' and '172' in row[model_column] else (
                '150' if row[make_column] == 'cessna' and '150' in row[model_column] else None
            )
        )
    ),
    axis=1
)

airplane_df['model_new'].value_counts()

172      6702
pa-28    4808
150      3251
152      2497
Name: model_new, dtype: int64

In [35]:
airplane_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64253 entries, 0 to 88886
Data columns (total 33 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                64253 non-null  object 
 1   Investigation.Type      64253 non-null  object 
 2   Accident.Number         64253 non-null  object 
 3   Event.Date              64253 non-null  object 
 4   Location                64219 non-null  object 
 5   Country                 64161 non-null  object 
 6   Latitude                24164 non-null  object 
 7   Longitude               24157 non-null  object 
 8   Airport.Code            39181 non-null  object 
 9   Airport.Name            41179 non-null  object 
 10  Injury.Severity         64253 non-null  object 
 11  Aircraft.damage         64253 non-null  object 
 12  Aircraft.Category       64253 non-null  object 
 13  Registration.Number     63589 non-null  object 
 14  Make                    64253 non-null

In [36]:
# Now we create a new dataframe where we only examine the models we are interested in purchasing.
planes_of_interest = airplane_df.dropna(subset=['model_new'])
planes_of_interest.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17258 entries, 2 to 88877
Data columns (total 33 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                17258 non-null  object 
 1   Investigation.Type      17258 non-null  object 
 2   Accident.Number         17258 non-null  object 
 3   Event.Date              17258 non-null  object 
 4   Location                17253 non-null  object 
 5   Country                 17232 non-null  object 
 6   Latitude                5379 non-null   object 
 7   Longitude               5376 non-null   object 
 8   Airport.Code            11305 non-null  object 
 9   Airport.Name            11771 non-null  object 
 10  Injury.Severity         17258 non-null  object 
 11  Aircraft.damage         17258 non-null  object 
 12  Aircraft.Category       17258 non-null  object 
 13  Registration.Number     17186 non-null  object 
 14  Make                    17258 non-null